In [ ]:
python -m pip install boto3

In [ ]:
import boto3
from botocore.exceptions import ClientError
from datetime import datetime

# ===== CONFIGURAÇÕES =====
AWS_ACCESS_KEY = 'SUA_ACCESS_KEY'
AWS_SECRET_KEY = 'SUA_SECRET_KEY'
REGION = 'us-east-1'

ACCOUNT_ID = '123456789012'
BUDGET_NAME = 'orcamento-mensal-aws'
BUDGET_AMOUNT = '100.00'  # USD

# ===== CLIENTE =====
client = boto3.client(
    'budgets',
    region_name=REGION,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY
)

# ===== CRIAÇÃO DO BUDGET =====
try:
    client.create_budget(
        AccountId=ACCOUNT_ID,
        Budget={
            'BudgetName': BUDGET_NAME,
            'BudgetLimit': {
                'Amount': BUDGET_AMOUNT,
                'Unit': 'USD'
            },
            'TimeUnit': 'MONTHLY',
            'BudgetType': 'COST',
            'TimePeriod': {
                'Start': datetime(2025, 1, 1)
            }
        }
        #NotificationsWithSubscribers=[]
    )

    print(f" Budget '{BUDGET_NAME}' criado com sucesso!")

except ClientError as e:
    error_code = e.response['Error']['Code']
    error_msg = e.response['Error']['Message']

    if error_code == 'DuplicateRecordException':
        print(f" O budget '{BUDGET_NAME}' já existe.")
    elif error_code == 'AccessDeniedException':
        print(" Permissão negada: verifique a policy IAM.")
    elif error_code == 'InvalidParameterException':
        print(" Parâmetros inválidos no budget.")
    else:
        print(" Erro inesperado:")
        print(error_msg)

except Exception as e:
    print(" Erro geral:")
    print(str(e))


In [ ]:
{
  "Effect": "Allow",
  "Action": [
    "budgets:CreateBudget",
    "budgets:DescribeBudgets",
    "budgets:DeleteBudget"
  ],
  "Resource": "*"
}